In [1]:
!pip install torch torchvision
!pip install transformers
!pip install tensorboardx
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.0 MB/s eta 0

In [2]:
import torch

torch.manual_seed(42)

In [3]:
import warnings

# mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')

warnings.filterwarnings('ignore')

Mounted at /content/gdrive


In [5]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from scipy.special import softmax


df_train = pd.read_csv('gdrive/MyDrive/data/comparg_train.tsv', sep='\t', encoding='utf-8')
df_test = pd.read_csv('gdrive/MyDrive/data/comparg_test.tsv', sep='\t', encoding='utf-8')

# df_train.head(25)

In [6]:
def mask_objects(row):
    row["text"] = row["answer"].replace(row["object_0"],"[FIRST_ENTITY]").replace(row["object_1"],"[SECOND_ENTITY]")
    return row

def transform_data(df):
  df = df.apply(mask_objects,axis=1)
  return df


In [ ]:
df_train = transform_data(df_train).sample(frac=1, random_state=42).reset_index(drop=True)
df_test = transform_data(df_test)

In [ ]:
args = {"overwrite_output_dir": True,
        "num_train_epochs": 10,
        "fp16": False,
        "train_batch_size": 16,
        "gradient_accumulation_steps":1,
        "evaluate_during_training": False,
        "max_seq_length": 64,
        "learning_rate": 3e-5,
        "no_cache": True,
        #"save_model_every_epoch": False,
        "reprocess_input_data": True,
        "output_dir": "gdrive/MyDrive/checkpoints"}

model = ClassificationModel("roberta", "roberta-large", num_labels=4, use_cuda=torch.cuda.is_available())
model.train_model(df_train, args=args)

_, raw_outputs, _ = model.eval_model(df_test)
pred_probs = softmax(raw_outputs, axis=1)
pred_labels = np.argmax(raw_outputs, axis=1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/5759 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/360 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/360 [00:00<?, ?it/s]

...

Running Epoch 7 of 10:   0%|          | 0/360 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/360 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/360 [00:00<?, ?it/s]

  0%|          | 0/1440 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/180 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_true=df_test.labels.tolist(), y_pred=pred_labels))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      1048
           2       0.85      0.83      0.84       273
           3       0.68      0.68      0.68       119

    accuracy                           0.90      1440
   macro avg       0.82      0.82      0.82      1440
weighted avg       0.90      0.90      0.90      1440



In [ ]:
test_model = ClassificationModel("roberta", "gdrive/MyDrive/checkpoints", num_labels=4, use_cuda=torch.cuda.is_available())

_, raw_outputs, _ = test_model.eval_model(df_test)
pred_probs_test = softmax(raw_outputs, axis=1) # [0.1, 0.1, 0.8]
pred_labels_test = np.argmax(raw_outputs, axis=1) # 3

print(classification_report(y_true=df_test.labels.tolist(), y_pred=pred_labels_test))

  0%|          | 0/1440 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/180 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      1048
           2       0.85      0.83      0.84       273
           3       0.68      0.68      0.68       119

    accuracy                           0.90      1440
   macro avg       0.82      0.82      0.82      1440
weighted avg       0.90      0.90      0.90      1440



In [ ]:
from sklearn.metrics import f1_score

def test_results(PATH_CHEKPOINTS):
  test_model = ClassificationModel("roberta", PATH_CHEKPOINTS, num_labels=4, use_cuda=torch.cuda.is_available())

  _, raw_outputs, _ = test_model.eval_model(df_test)
  pred_probs_test = softmax(raw_outputs, axis=1)
  pred_labels_test = np.argmax(raw_outputs, axis=1)

  pred_labels_test_transformed = ["NONE" if x == 0 else "BETTER" if x == 2 else "WORSE" for x in pred_labels_test]
  df_test_transformed = df_test.copy()
  df_test_transformed["labels"].replace({0: "NONE", 2: "BETTER", 3: "WORSE"}, inplace=True)

  print(classification_report(y_true=df_test_transformed.labels.tolist(), y_pred=pred_labels_test_transformed))
  print(f"F1-micro: {f1_score(y_true=df_test_transformed.labels.tolist(), y_pred=pred_labels_test_transformed, average='micro')}")

In [ ]:
PATH_CHEKPOINTS = "gdrive/MyDrive/checkpoints/checkpoint-1800-epoch-5"
test_results(PATH_CHEKPOINTS)

  0%|          | 0/1440 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/180 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      BETTER       0.87      0.83      0.85       273
        NONE       0.93      0.95      0.94      1048
       WORSE       0.73      0.65      0.68       119

    accuracy                           0.91      1440
   macro avg       0.84      0.81      0.83      1440
weighted avg       0.90      0.91      0.90      1440

F1-micro: 0.9055555555555556


In [ ]:
#df_test["pred_labels"] = pred_labels_test
#df_test["pred_probs"] = list(pred_probs_test)
#df_test.to_csv('gdrive/MyDrive/data/5_epochs_comparg_test.tsv', sep='\t', index=False)

In [ ]:
PATH_CHEKPOINTS = "gdrive/MyDrive/checkpoints/checkpoint-2880-epoch-8"
test_results(PATH_CHEKPOINTS)

  0%|          | 0/1440 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/180 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      BETTER       0.84      0.86      0.85       273
        NONE       0.96      0.93      0.95      1048
       WORSE       0.66      0.76      0.71       119

    accuracy                           0.91      1440
   macro avg       0.82      0.85      0.84      1440
weighted avg       0.91      0.91      0.91      1440



In [ ]:
PATH_CHEKPOINTS = "gdrive/MyDrive/checkpoints/checkpoint-3600-epoch-10"
test_results(PATH_CHEKPOINTS)

  0%|          | 0/1440 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/180 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      BETTER       0.85      0.83      0.84       273
        NONE       0.94      0.95      0.94      1048
       WORSE       0.68      0.68      0.68       119

    accuracy                           0.90      1440
   macro avg       0.82      0.82      0.82      1440
weighted avg       0.90      0.90      0.90      1440



In [ ]:
### Train on English, test on Russian

In [9]:
import pandas as pd

df_train_en = pd.read_csv('gdrive/MyDrive/data/comparg_train.tsv', sep='\t', encoding='utf-8')
df_test_en = pd.read_csv('gdrive/MyDrive/data/comparg_test.tsv', sep='\t', encoding='utf-8')
df_train = pd.concat([df_train_en, df_test_en])

df_test = pd.read_csv('gdrive/MyDrive/data/ru_sentences_with_objs.tsv', sep='\t', encoding='utf-8')
df_test = df_test.rename(columns={"text": "answer", "tag": "labels"})
df_test["labels"].replace({"NONE": 0, "BETTER": 2, "WORSE": 3}, inplace=True)
# df_test.head()

In [10]:
import string

def mask_objects_ru(row):
    object_1 = row["Object1"].translate(str.maketrans('', '', string.punctuation))
    object_2 = row["Object2"].translate(str.maketrans('', '', string.punctuation))

    if len(object_1.split()) == 1 and len(object_2.split()) == 1:
      row["text"] = " ".join(["[FIRST_ENTITY]" if object_1.lower() in w.lower() else "[SECOND_ENTITY]" if object_2.lower() in w.lower() else w for w in row["answer"].split()])

    elif len(object_1.split()) > 1 and len(object_2.split()) == 1:
      out = []
      for w in row["answer"].split():
        for o1 in object_1.lower().split():
          if o1 in w.lower() and "[FIRST_ENTITY]" not in out and w.lower() not in [w.lower() for w in out]:
            out.append("[FIRST_ENTITY]")
            break
          elif object_2.lower() in w.lower() and "[SECOND_ENTITY]" not in out:
            out.append("[SECOND_ENTITY]")
            break
          else:
            if w.lower() not in [w.lower() for w in out] and w.lower() not in object_1.lower().split() and w.lower() not in object_2.lower().split():
              out.append(w)
      row["text"] = " ".join(out)

    elif len(object_1.split()) == 1 and len(object_2.split()) > 1:
      out = []
      for w in row["answer"].split():
        for o2 in object_2.lower().split():
          if o2 in w.lower() and "[SECOND_ENTITY]" not in out and w.lower() not in [w.lower() for w in out]:
            out.append("[SECOND_ENTITY]")
            break
          elif object_1.lower() in w.lower() and "[FIRST_ENTITY]" not in out:
            out.append("[FIRST_ENTITY]")
            break
          else:
            if w.lower() not in [w.lower() for w in out] and w.lower() not in object_1.lower().split() and w.lower() not in object_2.lower().split():
              out.append(w)
      row["text"] = " ".join(out)

    else:
      row["text"] = row["answer"].replace(object_1,"[FIRST_ENTITY]").replace(object_2,"[SECOND_ENTITY]")
      # row["text"] = row["answer"].replace(row["object_0"],"[FIRST_ENTITY]").replace(row["object_1"],"[SECOND_ENTITY]")
    return row

def transform_data_ru(df):
  df = df.apply(mask_objects_ru,axis=1)
  return df

In [11]:
df_train = transform_data(df_train).sample(frac=1, random_state=42).reset_index(drop=True)
df_test = transform_data_ru(df_test)
# df_test.to_csv('gdrive/MyDrive/data/process_test.tsv', sep='\t', encoding='utf-8', index=False)

In [12]:
args = {"overwrite_output_dir": True,
        "num_train_epochs": 5,
        "fp16": False,
        "train_batch_size": 16,
        "gradient_accumulation_steps":1,
        "evaluate_during_training": False,
        "max_seq_length": 64,
        "learning_rate": 3e-5,
        "no_cache": True,
        "save_model_every_epoch": False,
        "reprocess_input_data": True,
        "output_dir": "gdrive/MyDrive/checkpoints"}

model = ClassificationModel("roberta", "xlm-roberta-large", num_labels=4, use_cuda=torch.cuda.is_available())
model.train_model(df_train, args=args)

_, raw_outputs, _ = model.eval_model(df_test)
pred_probs = softmax(raw_outputs, axis=1)
pred_labels = np.argmax(raw_outputs, axis=1)

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLMRobertaTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


  0%|          | 0/7199 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

...

Running Epoch 2 of 5:   0%|          | 0/450 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/450 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/450 [00:00<?, ?it/s]

  0%|          | 0/1208 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/151 [00:00<?, ?it/s]

In [13]:
print(classification_report(y_true=df_test.labels.tolist(), y_pred=pred_labels))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90       850
           2       0.81      0.60      0.69       257
           3       0.60      0.51      0.55       101

    accuracy                           0.84      1208
   macro avg       0.76      0.69      0.72      1208
weighted avg       0.83      0.84      0.83      1208



In [78]:
from sklearn.metrics import f1_score

def test_results(PATH_CHEKPOINTS):
  test_model = ClassificationModel("roberta", PATH_CHEKPOINTS, num_labels=4, use_cuda=torch.cuda.is_available())

  _, raw_outputs, _ = test_model.eval_model(df_test)
  pred_probs_test = softmax(raw_outputs, axis=1)
  pred_labels_test = np.argmax(raw_outputs, axis=1)

  pred_labels_test_transformed = ["NONE" if x == 0 else "BETTER" if x == 2 else "WORSE" for x in pred_labels_test]
  df_test_transformed = df_test.copy()
  df_test_transformed["labels"].replace({0: "NONE", 2: "BETTER", 3: "WORSE"}, inplace=True)

  print(classification_report(y_true=df_test_transformed.labels.tolist(), y_pred=pred_labels_test_transformed))
  print(f"F1-micro: {f1_score(y_true=df_test_transformed.labels.tolist(), y_pred=pred_labels_test_transformed, average='micro')}")

In [81]:
PATH_CHEKPOINTS = "gdrive/MyDrive/checkpoints/checkpoint-1800-epoch-5"
test_results(PATH_CHEKPOINTS)

  0%|          | 0/1208 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/151 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      BETTER       0.84      0.57      0.68       257
        NONE       0.86      0.94      0.90       850
       WORSE       0.52      0.55      0.54       101

    accuracy                           0.83      1208
   macro avg       0.74      0.69      0.71      1208
weighted avg       0.83      0.83      0.82      1208

F1-micro: 0.8302980132450332


In [14]:
df_test["pred_labels"] = pred_labels
df_test["pred_probs"] = list(pred_probs)
df_test.to_csv('gdrive/MyDrive/data/5_epochs_ru_test_trained_full.tsv', sep='\t', index=False)